In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests

from time import sleep
import pandas as pd 
import numpy as np


In [498]:
driver = webdriver.Chrome("/Applications/chromedriver")

In [499]:

store_url = 'https://www.coursesu.com/drive-hyperu-murserigne'
driver.get(store_url)




In [239]:
main_menus = driver.find_elements_by_class_name('main-menu-item')[3:11]

sub_menus = [cat.find_elements_by_class_name('navigation__sub-menu') for cat in main_menus]
sub_menus = [item for items in sub_menus for item in items]

category_urls = []
categories = [links.find_elements_by_class_name('navigation__item--third') for links in sub_menus]
for elements in categories:
    if len(elements) > 0:
        for index in range(len(elements)):
            if index == 0:
                pass
            else:
                category_urls.append(elements[index].get_attribute("href"))



In [500]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}
product_pages = []

for link in category_urls:
    driver.get(link)
    sleep(1)
    shelves_breadcrumb = driver.find_elements_by_class_name('breadcrumb')[0].text
    main_shelve = shelves_breadcrumb.split('\n')[0]
    parent_shelve = shelves_breadcrumb.split('\n')[1]
    shelve = shelves_breadcrumb.split('\n')[2]

    for i in range(5):
        try:
            actions = ActionChains(driver)
            actions.move_to_element(driver.find_element_by_class_name("infinite-scroll-placeholder")).perform()
        except:
            pass

    results = []
    soup = BeautifulSoup(driver.page_source, 'lxml')
    results.append(soup.select('.product-tile'))
    results = results[0]

    filter_btn = driver.find_element_by_class_name('filter-suggestions-section a')
    filter_panel = driver.execute_script("arguments[0].click();", filter_btn)
    soup_filters = BeautifulSoup(driver.page_source, 'lxml')
    try:
        result_filters = soup_filters.select('.simplebar-content')[0]
        try:
            brands = [label['for'] for label in result_filters.find('ul', class_="brand").find_all('label')]
        except:
            brands = ''
    except:
        pass

    product_pages.append({"url":link, 
                          "shelter_main": main_shelve,
                          "shelter_parent": parent_shelve,
                          "shelter": shelve,
                          "brands": brands,
                          "soup": results
                          })

len(product_pages)

359

In [502]:
data = pd.DataFrame(product_pages)
data.tail()

,url,shelter_main,shelter_parent,sheter,brands,soup
354,https://www.coursesu.com/c/vins-bieres-alcools...,"Vins, bières, alcools",Vins doux,France,"[DAURE, PRODUIT 1ER PRIX, PRODUIT U, RAPHA, RE...","[[ dwMarker=""product"" dwContentID=""65a3fc18c79..."
355,https://www.coursesu.com/c/vins-bieres-alcools...,"Vins, bières, alcools",Vins doux,Autres,"[BYRRH, CRUZ, DON SIMON, MARTINI, PENASOL, PRO...","[[ dwMarker=""product"" dwContentID=""1a1b2bc911c..."
356,https://www.coursesu.com/c/vins-bieres-alcools...,"Vins, bières, alcools",Autres digestifs,"Amer, bitter, gentiane","[PICON, PRODUIT U, SANS MARQUE]","[[ dwMarker=""product"" dwContentID=""2382842826d..."
357,https://www.coursesu.com/c/vins-bieres-alcools...,"Vins, bières, alcools",Autres digestifs,Sans alcool,"[MISTER COCKTAIL, PRODUIT 1ER PRIX, PRODUIT U]","[[ dwMarker=""product"" dwContentID=""f305ce60c8c..."
358,https://www.coursesu.com/c/vins-bieres-alcools...,"Vins, bières, alcools",Autres digestifs,Digestifs,"[LA MAZIERE, PRODUIT 1ER PRIX]","[[ dwMarker=""product"" dwContentID=""8228790ec83..."


In [504]:
products = []
bad_products = []

for page in product_pages:
    shelter_main = page['shelter_main']
    shelter_parent = page['shelter_parent']
    shelter = page['shelter']    
    brands = [brand for brand in page['brands']]
    
    for product in page['soup']:
        product_name = product.find("h3", class_="product-name").find("a")["data-copy-title"]
        product_link = product.find("div", class_="product-image").find("a")["href"]
        product_id = product['data-itemid']
        try:
            product_image = product.find("div", class_="product-image").find("picture").find("img")["src"]
        except:
            product_image = product.find("div", class_="product-image").find("img")["data-src"]
        try:
            product_origin = product.find("div", class_="product-origine").text[1:-1].lower().capitalize()
        except:
            pass
        try:            
            product_price = product.find("div", class_="product-pricing").find('span', class_="sale-price")["data-item-price"]
        except:
            bad_products.append(product)
            break
        product_price_per_unit = product.find("div", class_="product-pricing").find('span', class_="unit-info").text            
        store_name = "Système U"
        if len(brands) > 0:
            brand = ''.join([brand for brand in brands if brand.lower() in product_name.lower()])
        else:
            brand = ""  
        clean_description = product_name.lower().strip().replace(brand.lower().strip(), '').strip()

        products.append({"shelter_main": shelter_main,
                         "shelter_parent": shelter_parent,
                         "shelter_child": shelter,
                         "brand": brand,  
                         "description1": clean_description,                             
                         "description": product_name,
                         "price": product_price,
                         "price_per_unit": product_price_per_unit,
                         #"promo_price": promo_price,
                         "store": store_name,
                         "image_url": product_image,                    
                         "url": product_link,
                         "ean": product_id
                        })
print(len(products))
print(len(bad_products))



8425
41


In [505]:
product_data = pd.DataFrame(products)
product_data = product_data.drop_duplicates()

export_csv = product_data.to_csv (r'/Users/vincentsalamand/Downloads/products_systemeu.csv', index = None, header=True)


In [506]:
# Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = product_data[product_data.duplicated()]
 
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [shelter_main, shelter_parent, shelter_child, brand, description1, description, price, price_per_unit, store, image_url, url, ean]
Index: []


In [514]:
product_data.head(2130)

,shelter_main,shelter_parent,shelter_child,brand,description1,description,price,price_per_unit,store,image_url,url,ean
0,Fruits et Légumes,Fruits,Fruits bio,,"banane cavendish, bio, catégorie 2, république...","Banane Cavendish, BIO, catégorie 2, République...",2.3,"2,30 €/kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/banane-cavendish-bio-categorie-2-republique...,2890030
1,Fruits et Légumes,Fruits,Fruits bio,,"orange salutiana, u bio, calibre 5/6, catégori...","Orange Salutiana, U BIO, calibre 5/6, catégori...",3.95,"2,64 €/kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/orange-salutiana-u-bio-calibre-5-6-categori...,5927091
2,Fruits et Légumes,Fruits,Fruits bio,,"orange naveline, bio, calibre 4, catégorie 2, ...","Orange naveline, BIO, calibre 4, catégorie 2, ...",0.86,"3,45 €/kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/orange-naveline-bio-calibre-4-categorie-2-e...,1396111
3,Fruits et Légumes,Fruits,Fruits bio,,"mandarine clemenvilla, u bio, calibre 1x-xx, c...","Mandarine Clemenvilla, U BIO, calibre 1X-XX, c...",3.95,"3,95 €/kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/mandarine-clemenvilla-u-bio-calibre-1x-xx-c...,5894870
4,Fruits et Légumes,Fruits,"Bananes, Kiwis, Fruits exotiques",,"banane cavendish, bio, catégorie 2, république...","Banane Cavendish, BIO, catégorie 2, République...",2.3,"2,30 €/kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/banane-cavendish-bio-categorie-2-republique...,2890030
...,...,...,...,...,...,...,...,...,...,...,...,...
2125,Produits frais,Charcuterie,"Lardons, poitrine, graisse à cuire",,lardons allumettes fumés à teneur réduite en s...,Lardons allumettes fumés à teneur réduite en s...,1.79,"11,94 €/kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/lardons-allumettes-fumes-a-teneur-reduite-e...,4680664
2126,Produits frais,Charcuterie,"Lardons, poitrine, graisse à cuire",,emincés de poulet traité en salaison cuite et ...,Emincés de poulet traité en salaison cuite et ...,1.95,"13,00 €/kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/eminces-de-poulet-traite-en-salaison-cuite-...,5393356
2127,Produits frais,Charcuterie,"Lardons, poitrine, graisse à cuire",,"emincés de filet de poulet a la provençale u, ...","Emincés de filet de poulet a la provençale U, ...",2.07,"13,80 €/kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/eminces-de-filet-de-poulet-a-la-provencale-...,2851472
2128,Produits frais,Charcuterie,"Lardons, poitrine, graisse à cuire",,foies de volaille confits cuits dans la graiss...,Foies de volaille confits cuits dans la graiss...,2.24,"7,47 €/kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/foies-de-volaille-confits-cuits-dans-la-gra...,1082736
